In [1]:
import torch
from torch import optim, nn
from torch.autograd import Variable
# import pixiedust

In [2]:
x = torch.tensor([[0,0,0,0],[1,0,0,0],[0,1,0,0],[0,0,1,0],[1,1,0,0],[1,0,1,0],[0,1,1,0],[1,1,1,0],[0,0,0,1],[1,0,0,1],[0,1,0,1],[0,0,1,1],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1.]])
x.requires_grad_()
target = torch.tensor([[0],[0],[0],[0],[0],[0],[0],[0],[1],[1],[1],[1],[1],[1],[1],[1.]])


#   Variables for performance metrics
epochs = 20
lr = 0.2
counter = 0

# Define 2 chained models
models = [
    nn.Sequential(
        nn.Linear(4, 3),
        nn.Tanh()
    ),
    nn.Sequential(
        nn.Linear(3, 1),
        nn.Sigmoid()
    )
]

# Create optimisers for each segment and link to their segment
optimizers = [
    optim.SGD(params=model.parameters(),lr=lr)
    for model in models
]

In [3]:
def train():
    # Training Logic
    for iter in range(epochs):

        # 1) erase previous gradients (if they exist)
        for opt in optimizers:
            opt.zero_grad()

        # 2) make a prediction
        a  = models[0](x)

        # 3) send the activation signal to the next model
        remote_a = a.detach()
        # re-enable autograd here
        remote_a.requires_grad_()

        pred =  models[1](remote_a)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()
        
        # 5) Backprop gradient to model behind
        grad_a = remote_a.grad.clone()
        
        # 5) Go backward from grad_a 
#         print(models[0][0].weight.grad)
        a.backward(grad_a)
#         print(models[0][0].weight.grad)

        # 5) change the weights
        for opt in optimizers:
            opt.step()

        # 6) print our progress
        # Do not use .data
        print(loss.detach())
        
train()

tensor(4.6285)
tensor(3.7840)
tensor(3.2810)
tensor(2.7966)
tensor(2.2425)
tensor(1.6886)
tensor(1.2308)
tensor(0.9033)
tensor(0.6829)
tensor(0.5345)
tensor(0.4317)
tensor(0.3582)
tensor(0.3037)
tensor(0.2621)
tensor(0.2297)
tensor(0.2037)
tensor(0.1826)
tensor(0.1651)
tensor(0.1505)
tensor(0.1380)
